In [ ]:
import os
import pathlib

import pytesseract
import cv2
import numpy as np

pytesseract.pytesseract.tesseract_cmd = (
    os.popen("brew list tesseract | grep 'bin'").read().strip()
)

img = cv2.imread(f"{str(pathlib.Path(__file__).parent.resolve())}/ocr_test_imgs/ocr_test_5.png")

def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

img = get_grayscale(img)

def adj_bright_contrast(image, alpha, beta):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    thresh, _ = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED) 

img = adj_bright_contrast(img, 1, 30)
img = thresholding(img)

img_str = pytesseract.image_to_string(img)

print(img_str)

In [ ]:
# TODO 1: crop images
# TODO 2: save cropped images in efficent way
# TODO 3: run ocr on cropped images
# TODO 4: store cropped images metadata with ocr texr in efficent way 
# TODO 5: search engine implementation 
# TODO 6: search gui implementation (https://www.anycodings.com/questions/matplotlib-pyplot-imshow-puts-image-in-button-instead-of-where-it-needs-to-be)